In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

In [8]:
import re

In [2]:
AllData = pd.read_csv('AllData.csv')

In [11]:
CaucusFiltered = pd.read_csv('CaucusFilter.csv')

In [9]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

def preprocess_text(text):
    text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text)

    tokens = word_tokenize(text)

    tokens = [word.lower() for word in tokens]

    tokens = [word for word in tokens if word.isalpha()]

    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if not word in stop_words]

    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    return ' '.join(tokens)

AllData['Proccessed Press Release'] = AllData['Press Release'].apply(preprocess_text)

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/aaryakagalwala/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/aaryakagalwala/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/aaryakagalwala/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [12]:
CaucusFiltered['Proccessed Press Release'] = CaucusFiltered['Press Release'].apply(preprocess_text)

In [14]:
AllData.to_csv('CleanedData.csv',index=False)
CaucusFiltered.to_csv('CleanedCaucus.csv',index=False)

In [21]:
CaucusFiltered['Caucus'].value_counts()

Congress Progressive      76
New Democrat Coalition    47
Freedom Party             34
Main Street               27
Name: Caucus, dtype: int64

In [16]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [19]:
def tokenize_by_sentence(text):
    doc = nlp(text)
    sentences = [sent.text for sent in doc.sents]
    return sentences

CaucusFiltered['sentences'] = CaucusFiltered['Proccessed Press Release'].apply(tokenize_by_sentence)
Caucus_sentences = CaucusFiltered.explode('sentences')
Caucus_sentences['sentence_index'] = Caucus_setences.groupby(level=0).cumcount()
Caucus_sentences.reset_index(drop=True,inplace=True)


KeyError: 'Processed Press Release'